# Dataset Labeling

**Mark Edward M. Gonzales, Lorene C. Uy, and Jacob Adrianne L. Sy (CSC713M)**<br>
mark_gonzales@dlsu.edu.ph, lorene_c_uy@dlsu.edu.ph, jacob_adrianne_l_sy@dlsu.edu.ph

In partial fulfillment of the requirements for the Machine Learning graduate class (CSC713M) under **Dr. Macario O. Cordel, II** of the Department of Computer Technology, College of Computer Studies, De La Salle University, this notebook details the process and presents the code for the **dataset labeling** stage of the investigatory project titled "Automatic Recommendation of Distance Metric for $k$-Means Clustering: A Meta-Learning Approach."

<hr>

# PART I: Preliminaries

The following libraries and modules — most of which are automatically bundled with an Anaconda installation — were used in this notebook:

Library/Module | Description | License
:-- | :-- | :--
<a href = "https://docs.python.org/3/library/os.html">`os`</a> | Provides miscellaneous operating system interfaces | Python Software Foundation License
<a href = "https://docs.python.org/3/library/shutil.html">`shutil`</a> | Provides high-level operations on files and collections of files | Python Software Foundation License
<a href = "https://docs.python.org/3/library/json.html">`json`</a> | Provides methods for encoding and decoding JavaScript Object Notation files | Python Software Foundation License
<a href = "https://docs.python.org/3/library/math.html">`math`</a> | Provides access to the mathematical functions defined by the C standard | Python Software Foundation License
<a href = "https://docs.python.org/3/library/warnings.html">`warnings`</a> | Provides control over warning messages | Python Software Foundation License
<a href = "https://pandas.pydata.org/">`pandas`</a> | Provides functions for data analysis and manipulation	 | BSD 3-Clause "New" or "Revised" License
<a href = "https://numpy.org/">`numpy`</a> | Provides a multidimensional array object, various derived objects, and an assortment of routines for fast operations on arrays | BSD 3-Clause "New" or "Revised" License
<a href = "https://www.scipy.org/">`scipy`</a> | Provides efficient numerical routines, such as those for numerical integration, interpolation, optimization, linear algebra, and statistics | BSD 3-Clause "New" or "Revised" License
<a href = "https://pyclustering.github.io/docs/0.8.2/html/index.html">`pyclustering`</a> | Collection of cluster analysis, graph coloring, travelling salesman problem algorithms, oscillatory and neural network models, containers, tools for visualization and result analysis | BSD 3-Clause "New" or "Revised" License
<a href = "https://scikit-learn.org/stable/">`scikit-learn`</a> | Python module for machine learning and predictive data analysis | BSD 3-Clause "New" or "Revised" License

*The descriptions were lifted from their respective websites.*
<br><br>

<div class="alert alert-block alert-info">
<b>Note:</b>  The pyclustering library is not included in Anaconda by default. The simplest way to install the library is to use pip by running the following command on the command prompt: <br>

**`pip3 install pyclustering`**
</div>

In [ ]:
from os import listdir

import json
import re
import warnings
import time

import pandas as pd
import numpy as np

from pyclustering.cluster.kmeans import kmeans
from pyclustering.cluster.center_initializer import random_center_initializer
from pyclustering.utils.metric import distance_metric
from pyclustering.utils.metric import type_metric

from scipy.spatial import distance
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

<div class="alert alert-block alert-warning">
If some of the methods in this notebook are not running (especially those related to <a href="https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics">sklearn.metrics</a>, it may be necessary to update the installed versions of these libraries. A quick way to do so is by running the following command on the Anaconda prompt: <br>

**`conda install anaconda`**
</div>

<hr>

## PART II: Initial Clustering of the Datasets

This section details the initial clustering of the datasets using the nine (9) distance metrics considered in [1].

## A. Distance Metrics


Let $o_i$ and $o_j$ be two observations (data points) in the dataset $d = \{a_{n \times m}\}$, where $n$ is the number of observations and $m$ is the number of features. 

### 1. Manhattan Distance

$$\text{dist}_{\text{manhattan}}(o_i, o_j) = \sum_{k = 1}^{m} |a_{ik} - a_{jk}|$$

`pyclustering` provides a built-in [implementation](https://pyclustering.github.io/docs/0.9.0/html/da/d3a/classpyclustering_1_1utils_1_1metric_1_1type__metric.html#a49a236683bbe19f1771fb16b9f68e144) of the Manhattan distance.

### 2. Euclidean Distance

$$\text{dist}_{\text{euclidean}}(o_i, o_j) = \sqrt{\sum_{k = 1}^{m} \left(a_{ik} - a_{jk}\right)^2}$$

`pyclustering` provides a built-in [implementation](https://pyclustering.github.io/docs/0.9.0/html/da/d3a/classpyclustering_1_1utils_1_1metric_1_1type__metric.html#ada41bdfbf917a3463b48f75c9a6ccae1) of the Euclidean distance.

### 3. Chebyshev Distance

$$\text{dist}_{\text{chebyshev}}(o_i, o_j) = \max_{0<k<m+1}\{|a_{ik} - a_{jk}|\}$$

`pyclustering` provides a built-in [implementation](https://pyclustering.github.io/docs/0.9.0/html/da/d3a/classpyclustering_1_1utils_1_1metric_1_1type__metric.html#ad4419ac00c1f1c484f86d0af82bf0164) of the Chebyshev distance.

### 4. Standardized Euclidean Distance

$$\text{dist}_{\text{standardized euclidean}}(o_i, o_j) = \sqrt{\sum_{k = 1}^{m} \frac{\left(a_{ik} - a_{jk}\right)^2}{s_k^2}}$$

where $s_k^2$ is the variance of $a_{\cdot k}$.

This method returns the standardized Euclidean distance between two observations.

**Parameters**:
- `u`: First observation 
- `v`: Second observation

**Return Value**:
- Standardized Euclidean distance between two observations

In [3]:
def standardized_euclidean(u, v):
    return distance.cdist([u], [v], 'seuclidean')[0][0]

**Sanity Check**: The standardized Euclidean distance between these two vectors should be 2.4495.

In [4]:
standardized_euclidean([0.1, 0.7, 0.3], [0.4, 0.6, 0.9])

2.449489742783178

### 5. Canberra Distance
$$\text{dist}_{\text{canberra}}(o_i, o_j) = \sum_{k = 1}^{m} \frac{|a_{ik} - a_{jk}|}{|a_{ik}| + |a_{jk}|}$$

`pyclustering` provides a built-in [implementation](https://pyclustering.github.io/docs/0.9.0/html/da/d3a/classpyclustering_1_1utils_1_1metric_1_1type__metric.html#a98e4af29af65e57b6dfbabcbd94c3be7) of the Canberra distance.

### 6. Mahalanobis Distance
$$\text{dist}_{\text{mahalanobis}}(o_i, o_j) = \sqrt{(o_i - o_j)^TM^{-1}(o_i - o_j)}$$

where $M$ is the covariance of $d$.

This method returns the Mahalanobis distance between two observations. Note that `iv` is the inverse covariance matrix of the dataset; it is a global variable that will be set in the method `label_dataset`.

**Parameters**:
- `u`: First observation 
- `v`: Second observation

**Return Value**:
- Mahalanobis distance between two observations

In [5]:
def mahalanobis(u, v):
    return distance.mahalanobis(u, v, iv)

### 7. Cosine Distance
$$\text{dist}_{\text{cosine}}(o_i, o_j) = 1 - \frac{o_i \cdot o_j}{||o_i|| \cdot ||o_j||}$$

`scipy` provides a built-in [implementation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cosine.html#scipy.spatial.distance.cosine) of the cosine distance.

### 8. Adjusted Cosine Distance
$$\text{dist}_{\text{adjusted cosine}}(o_i, o_j) = 1 - \frac{\sum_{k=1}^m \left(a_{jk} - \text{mean}(a_{\cdot k})\right)\left(a_{jk} - \text{mean}(a_{\cdot k})\right)}{\sqrt{\sum_{k=1}^m \left(a_{ik} - \text{mean}(a_{\cdot k})\right)^2} \sqrt{\sum_{k=1}^m \left(a_{jk} - \text{mean}(a_{\cdot k})\right)^2}}$$

In [6]:
def adjusted_cosine(u, v):
    return distance.cosine(u - np.mean(u), v - np.mean(v))

**Sanity Check**: The standardized Euclidean distance between these two vectors should be 0.7832.

In [7]:
adjusted_cosine([0.1, 0.7, 0.3], [0.4, 0.6, 0.9])

0.78322250761897

### 9. Pearson Correlation Distance
$$\text{dist}_{\text{pearson}}(o_i, o_j) = 1 - \frac{\sum_{k=1}^m \left(a_{jk} - \text{mean}(a_{i \cdot})\right)\left(a_{jk} - \text{mean}(a_{j \cdot})\right)}{\sqrt{\sum_{k=1}^m \left(a_{ik} - \text{mean}(a_{i \cdot})\right)^2} \sqrt{\sum_{k=1}^m \left(a_{jk} - \text{mean}(a_{j \cdot})\right)^2}}$$

`scipy` provides a built-in [implementation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.correlation.html) of the cosine distance.

### Mapping the Distance Metrics to Functions

The dictionary below maps the distance metrics to the method implementing them.

In [8]:
metric_functions =  {'manhattan': distance_metric(type_metric.MANHATTAN),
                     'euclidean': distance_metric(type_metric.EUCLIDEAN),
                     'chebyshev': distance_metric(type_metric.CHEBYSHEV),
                     'standardized_euclidean': distance_metric(type_metric.USER_DEFINED, func=standardized_euclidean),
                     'canberra': distance_metric(type_metric.CANBERRA),
                     'mahalanobis': distance_metric(type_metric.USER_DEFINED, func=mahalanobis),
                     'cosine': distance_metric(type_metric.USER_DEFINED, func=distance.cosine),
                     'adjusted-cosine': distance_metric(type_metric.USER_DEFINED, func=adjusted_cosine), 
                     'pearson': distance_metric(type_metric.USER_DEFINED, func=distance.correlation)}

This method returns the function corresponding to the specified distance metric.

**Parameter**:
- `metric`: Distance metric

**Return**:
- Function corresponding to the specified distance metric

In [9]:
def get_metric_function(metric):
    return metric_functions[metric]

<hr>

## B. Helper Functions

This subsection focuses on the helper functions for returning the cluster assignment and the ground-truth labels of the observations.

### 1. Cluster assignment of the observations

This method returns a list representing the cluster assignment of the observations. Formally, it returns a list $A$ where $A[i]$ is the cluster to which the $i^{\text{th}}$ observation belongs.

**Parameters**:
- `num_observations`: Number of observations in the dataset
- `clusters`: Two-dimensional list where `clusters[i]` contains the indices of the observations that belong to the $i^{\text{th}}$ cluster

**Return Value**:
- A one-dimensional list $A$ where $A[i]$ is the cluster to which the $i^{\text{th}}$ observation belongs

In [10]:
def get_cluster_mapping(num_observations, clusters):
    cluster_mapping = [0 for _ in range(num_observations)]
    
    cluster_idx = 0
    for cluster in clusters:
        for x in cluster:
            cluster_mapping[x] = cluster_idx
            
        cluster_idx += 1
        
    return cluster_mapping

### 2. Ground-truth labels of the observations

Returns a list representing the ground-truth labels of the observations. Formally, it returns a list $A$ where $A[i]$ is the ground-truth label of the $i^{\text{th}}$ observation.

**Precondition**:
- The labels are found at the last column of the dataset.

**Parameter**:
- `data`: Dataset

**Return Value**:
- A one-dimensional list $A$ where $A[i]$ is the ground-truth label of the $i^{\text{th}}$ observation

In [11]:
def get_ground_truth(data):
    return data[data.columns[-1]].astype('category').cat.codes.to_numpy()

### 3. $k$-means clustering

Returns the cluster assignment of the observations after performing $k$-means clustering. Formally, it returns a two-dimensional list $A$ where $A[i]$ contains the indices of the observations that belong to the $i^{\text{th}}$ cluster. 

**Parameters**:
- `X`: Observations
- `k`: Number of clusters
- `distance_metric`: Distance measure to be used in clustering

**Return Value**:
- A two-dimensional list $A$ where $A[i]$ contains the indices of the observations that belong to the $i^{\text{th}}$ cluster

In [12]:
def k_means_cluster(X, k, distance_metric):
    initial_centers = random_center_initializer(X, k, random_state=96024).initialize()
    
    k_means = kmeans(X, initial_centers, metric=distance_metric)    
    k_means.process()
    clusters = k_means.get_clusters()
    
    return clusters

<hr>

## C. Dataset Labeling

Before performing $k$-means, a minimal suite of preprocessing techniques is performed:
- **Data imputation** (using the mean of the values for numerical features)
- **Data normalization** (following [2, 3]). Let $x$ be the value to be normalized and $x_{\text{max}}$ and $x_{\text{min}}$ be the maximum and minimum values:

   $$x_\text{normalized} = \frac{x - x_\text{min}}{x_\text{max} - x_\text{min}},$$

   thus $x_\text{normalized}$ will always fall in the interval $[0, 1]$.
   
These preprocessing techniques are performed via the [simple imputation transformer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) and the [min-max scaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) of `scikit-learn`.

For the evaluation, two cluster validity indices are used:
- **Adjusted Rand Index (ARI)**. An external measure, it compares the cluster assignments against the ground-truth assignments. The (unadjusted) Rand index is given by the formula

   $$\text{ARI} =\text{ } \frac{\text{TP}+\text{TN}}{\text{TP} + \text{TN} + \text{FP} + \text{FN}},$$
   
   where $\text{TP}$, $\text{TN}$, $\text{FP}$, and $\text{FN}$ refer to the number of true positives, true negatives, false positives, and false negatives.
   
   The adjusted Rand index is a corrected-for-chance version following the scheme introduced in [4]. 
  
  
- **Davies-Bouldin Index (DBI)**. An internal measure, it captures the average similarity score (ratio of within-cluster to between-cluster distances) of each cluster with the most similar cluster, as delineated in [5].

These scores are computed via the `scikit-learn` methods for calculating the [ARI]() and [DBI]().

The method below returns two dictionaries containing the evaluation results after performing $k$-means clustering using the distance measures of interest. The first dictionary uses the adjusted Rand Index as the cluster validation index while the second dictionary uses the Davies-Bouldin index.

**Parameter**:
- `dataset`: Dataset

**Return Values**:
- Dictionary containing the adjusted rand indices after performing $k$-means clustering using the distance measures of interest
- Dictionary containing the Davies-Bouldin indices after performing $k$-means clustering using the distance measures of interest

In [13]:
def label_dataset(dataset):
    NO_HEADER = r'noheader'
    
    # Denotes that using the distance metric results in the k-means clustering 
    # to fail to converge or the execution to run into a mathematical error
    # (e.g., division by zero)
    NOT_APPLICABLE = "NOT_APPLICABLE"
    
    # Some of the datasets are not encoded in the default UTF-8.
    if re.search(NO_HEADER, dataset):
        data_raw = pd.read_csv(dataset, encoding='latin-1', header = None)
    else:
        data_raw = pd.read_csv(dataset, encoding='latin-1')
        
    data_raw[data_raw.columns[-1]] = data_raw[data_raw.columns[-1]].fillna(data_raw[data_raw.columns[-1]].mode()[0])
        
    data = data_raw.to_numpy()    
    X, y = np.split(data, [-1], axis=1)
    
    # Data imputation
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    X = imp.fit_transform(X)
    
    # Min-max normalization to [0, 1]
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    
    # Dictionaries for storing the evaluation results
    eval_results_ari = {}        # For adjusted Rand index
    eval_results_dbi = {}        # For Davies-Bouldin index
    
    time_elapsed = {}
    
    X_tolist = X.tolist()
    num_clusters = len(np.unique(y))
    
    # Iterate through all the distance metrics.
    for distance_metric in metric_functions:
        # Compute the inverse of the covariance matrix for Mahalanobis distance.
        if distance_metric == 'mahalanobis': 
            global iv
            
            # Taken from https://github.com/scipy/scipy/blob/v1.8.0/scipy/spatial/distance.py#L246-L247
            iv = np.linalg.pinv(np.atleast_2d(np.cov(X.astype(np.double, copy=False).T))).T.copy()
        
        metric_not_applicable = False
        with warnings.catch_warnings(record=True) as w:
            # Perform k-means clustering.
            try:
                clusters = k_means_cluster(X_tolist, num_clusters, get_metric_function(distance_metric))
            except RuntimeWarning:
                # Performing k-means runs into an error.
                metric_not_applicable = True
            
            # Performing k-means runs into an error.
            if len(w) > 0:
                metric_not_applicable = True
        
        # Get the cluster assignments and ground-truth labels.
        start_time = time.perf_counter()
        cluster_mapping = get_cluster_mapping(len(X), clusters)
        end_time = time.perf_counter()
        
        time_elapsed[distance_metric] = end_time - start_time
        
        ground_truth = get_ground_truth(data_raw)
                
        if metric_not_applicable:
            eval_results_ari[distance_metric] = NOT_APPLICABLE
            eval_results_dbi[distance_metric] = NOT_APPLICABLE
        else:
            # Evaluate the results of clustering.
            eval_results_ari[distance_metric] = metrics.adjusted_rand_score(ground_truth, cluster_mapping)
            
            if len(np.unique(cluster_mapping)) < 2:
                eval_results_dbi[distance_metric] = NOT_APPLICABLE
            else:
                eval_results_dbi[distance_metric] = metrics.davies_bouldin_score(X, cluster_mapping)
                    
    return eval_results_ari, eval_results_dbi, time_elapsed

The function below is a utility method for logging the results of the $k$-means clustering to text files. For each dataset, three output files are produced, corresponding to the *(i)* performance based on ARI, *(ii)* performance based on DBI, and *(iii)* the time taken to perform $k$-means using each distance metric.

**Parameter**:
- `path`: Path to the text file
- `str_data`: Results of the $k$-means clustering

In [14]:
def labels_to_txt(path, str_data):
    f = open(f"{path}", "w")
    f.write(str_data)
    f.close()

The code below calls the previous methods to perform $k$-means clustering on the datasets and log the results to text files. It assumes that the datasets are stored in the folder `final_datasets` and that the output files will be saved inside the folder `dataset_labels/txt_indiv`. 

In [15]:
folder = 'new_folder'
datasets = listdir(folder)

warnings.filterwarnings("error")

# Number of batches into which the datasets will be split
NUMBER_OF_BATCHES = 5

batched_datasets = np.array_split(datasets, NUMBER_OF_BATCHES)

# Current batch to be processed
current_batch = 0

for i, datasets in enumerate(batched_datasets[current_batch:]):
    for dataset in datasets:    
        # Label the dataset with the pertinent distance metrics.
        ari, dbs, time_elapsed = label_dataset(f"{folder}/{dataset}")

        # Create file paths for the logging the results.
        filename, ext = dataset.rsplit('.', 1)
        ari_path =  f"./dataset_labels/txt_indiv/{filename}_ari.txt"
        dbs_path = f"./dataset_labels/txt_indiv/{filename}_dbs.txt"
        time_path = f"./dataset_labels/txt_indiv/{filename}_time.txt"
        
        # Save the results to text files.
        labels_to_txt(ari_path, str(ari))
        labels_to_txt(dbs_path, str(dbs))
        labels_to_txt(time_path, str(time_elapsed))

./dataset_labels/txt_indiv/online_shoppers_intention1_one_hot_time.txt
./dataset_labels/txt_indiv/online_shoppers_intention_one_hot_time.txt


The code below consolidates the [output text files](https://github.com/memgonzales/meta-learning-clustering/tree/master/dataset_labels) into a [single CSV file](https://github.com/memgonzales/meta-learning-clustering/blob/master/dataset_labels/csv_collated/results_all_metrics.csv) with filename `results_all_metrics.csv` and with the following columns:
- Name of dataset
- Best distance metric based on ARI
- ARI value when the best distance betric based on ARI is used
- Best distance metric based on DBI
- DBI value when the best distance betric based on ARI is used
- Worst distance metric based on ARI
- ARI value when the worst distance betric based on ARI is used
- Worst distance metric based on DBI
- DBI value when the worst distance betric based on ARI is used

Note that:
- For ARI, higher values denote better clustering quality.
- For DBI, lower values denote better clustering quality.
- Since the study is a essentially a multiclass classification task, ties in the selection of the best metric are broken by choosing the distance metric that requires a lower runtime (measured empirically).

It is assumed that the output text files are in the folder `dataset_labels/txt_indiv` and that the consolidated CSV file will be stored in the folder `dataset_labels/csv_collated`.

In [69]:
folder = "dataset_labels/txt_indiv"
datasets = listdir(folder)

NOT_APPLICABLE = "NOT_APPLICABLE"

# Number of output text files for each dataset (ARI, DBS, and time elapsed)
CHUNKED_BY = 3 
chunked_datasets=[datasets[i:i + CHUNKED_BY] for i in range(0, len(datasets), CHUNKED_BY)]

HEADER = "dataset,best_dist_metric_ari,best_dist_metric_ari_eval,best_dist_metric_dbs,best_dist_metric_dbs_eval,worst_dist_metric_ari,worst_dist_metric_ari_eval,worst_dist_metric_dbs,worst_dist_metric_dbs_eval\n"
f_out = open('./dataset_labels/csv_collated/results_all_metrics.csv', 'w')
f_out.write(HEADER)

for dataset in chunked_datasets:
    ari, dbs, time_elapsed = dataset[0], dataset[1], dataset[2]

    # Convert text files to dictionaries.
    f_ari = open(f"./dataset_labels/txt_indiv/{ari}", "r")
    ari_dict = f_ari.read()
    ari_dict = ari_dict.replace("'", '"')
    ari_dict = json.loads(ari_dict)
    f_ari.close()

    f_dbs = open(f"./dataset_labels/txt_indiv/{dbs}", "r")
    dbs_dict = f_dbs.read()
    dbs_dict = dbs_dict.replace("'", '"')
    dbs_dict = json.loads(dbs_dict)
    f_dbs.close()

    f_time = open(f"./dataset_labels/txt_indiv/{time_elapsed}", "r")
    time_dict = f_time.read()
    time_dict = time_dict.replace("'", '"')
    time_dict = json.loads(time_dict)
    f_time.close()

    ari_sorted = []
    dbs_sorted = []

    # Convert dictionaries to arrays of the form
    # [key, perfomance, time].
    for key in ari_dict.keys():
        if ari_dict[key] != NOT_APPLICABLE:
            ari_sorted.append([key, ari_dict[key], time_dict[key]])
        if dbs_dict[key] != NOT_APPLICABLE:
            dbs_sorted.append([key, dbs_dict[key], time_dict[key]])
                    
    # Sort arrays by performance (best to worst) then by time (fastest to slowest).
    ari_sorted.sort(key=lambda items: (items[1], -items[2]), reverse = True)
    dbs_sorted.sort(key=lambda items: (items[1], items[2]))

    filename, _ = ari.rsplit('_', 1)

    # Get the best distance metric based on ARI
    ari_len = len(ari_sorted)
    if ari_len != 0:
        best_dist_metric_ari = ari_sorted[0][0]
        best_dist_metric_ari_eval = ari_sorted[0][1]
    else:
        best_dist_metric_ari = NOT_APPLICABLE
        best_dist_metric_ari_eval = NOT_APPLICABLE
    
    # Get the best distance metric based on DBS
    dbs_len = len(dbs_sorted)
    if dbs_len != 0:
        best_dist_metric_dbs = dbs_sorted[0][0]
        best_dist_metric_dbs_eval = dbs_sorted[0][1]
    else:
        best_dist_metric_dbs = NOT_APPLICABLE
        best_dist_metric_dbs_eval = NOT_APPLICABLE

    # Get the worst distance metric based on ARI
    if ari_len != 0:
        worst_dist_metric_ari = ari_sorted[ari_len - 1][0]
        worst_dist_metric_ari_eval = ari_sorted[ari_len - 1][1]
    else:
        worst_dist_metric_ari = NOT_APPLICABLE
        worst_dist_metric_ari_eval = NOT_APPLICABLE

    # Get the worst distance metric based on DBS
    if dbs_len != 0:
        worst_dist_metric_dbs = dbs_sorted[dbs_len - 1][0]
        worst_dist_metric_dbs_eval = dbs_sorted[dbs_len - 1][1]
    else:
        worst_dist_metric_dbs = NOT_APPLICABLE
        worst_dist_metric_dbs_eval = NOT_APPLICABLE

    DATA = f"{filename},{best_dist_metric_ari},{best_dist_metric_ari_eval},{best_dist_metric_dbs},{best_dist_metric_dbs_eval},{worst_dist_metric_ari},{worst_dist_metric_ari_eval},{worst_dist_metric_dbs},{worst_dist_metric_dbs_eval}\n"
    f_out.write(DATA)

f_out.close()

<hr>

## PART III: Reclustering of the Datasets

In [62]:
clustering_results = pd.read_csv('dataset_labels/csv_collated/results_all_metrics.csv')
clustering_results

,dataset,best_dist_metric_ari,best_dist_metric_ari_eval,best_dist_metric_dbs,best_dist_metric_dbs_eval,worst_dist_metric_ari,worst_dist_metric_ari_eval,worst_dist_metric_dbs,worst_dist_metric_dbs_eval
0,a1_raw,chebyshev,0.199553,euclidean,1.563114,standardized_euclidean,-0.001595,mahalanobis,3.151034
1,a1_va3,cosine,0.237339,pearson,1.980815,standardized_euclidean,-0.000121,canberra,4.132754
2,a2_raw,canberra,0.359592,chebyshev,1.302162,standardized_euclidean,-0.001409,mahalanobis,1.948717
3,a2_va3,cosine,0.182077,pearson,1.860610,standardized_euclidean,-0.001797,canberra,3.355306
4,a3_raw,euclidean,0.287089,euclidean,1.359764,standardized_euclidean,-0.000378,mahalanobis,2.730803
...,...,...,...,...,...,...,...,...,...
335,winequality-white,euclidean,0.041872,euclidean,1.780022,standardized_euclidean,0.000000,canberra,3.004119
336,wine,canberra,0.046130,canberra,1.308862,mahalanobis,-0.007518,mahalanobis,1.920065
337,wisconsin,cosine,0.037550,manhattan,1.682554,canberra,-0.001336,mahalanobis,4.262601
338,Zemberek-Stemmed,cosine,0.541358,manhattan,0.870462,standardized_euclidean,0.000000,cosine,10.894565


In [63]:
clustering_results['best_dist_metric_ari'].value_counts()

manhattan                 90
mahalanobis               65
euclidean                 46
canberra                  41
chebyshev                 29
cosine                    24
adjusted-cosine           24
pearson                   11
standardized_euclidean    10
Name: best_dist_metric_ari, dtype: int64

In [64]:
clustering_results['best_dist_metric_dbs'].value_counts()

chebyshev          123
euclidean           88
manhattan           57
canberra            20
cosine              17
mahalanobis         17
pearson              9
adjusted-cosine      9
Name: best_dist_metric_dbs, dtype: int64

In [65]:
folder = "dataset_labels/txt_indiv"
datasets = listdir(folder)

NOT_APPLICABLE = "NOT_APPLICABLE"

TOP_N = 3

# Number of output text files for each dataset (ARI, DBS, and time elapsed)
CHUNKED_BY = 3 
chunked_datasets=[datasets[i:i + CHUNKED_BY] for i in range(0, len(datasets), CHUNKED_BY)]

HEADER = "dataset,best_dist_metric_ari,best_dist_metric_ari_eval,best_dist_metric_dbs,best_dist_metric_dbs_eval\n"
f_out = open('./dataset_labels/csv_collated/results_top_metrics.csv', 'w')
f_out.write(HEADER)

for dataset in chunked_datasets:
    ari, dbs, time_elapsed = dataset[0], dataset[1], dataset[2]

    # Convert text files to dictionaries.
    f_ari = open(f"./dataset_labels/txt_indiv/{ari}", "r")
    ari_dict = f_ari.read()
    ari_dict = ari_dict.replace("'", '"')
    ari_dict = json.loads(ari_dict)
    f_ari.close()
    
    for distance_metric in clustering_results['best_dist_metric_ari'].value_counts().index[TOP_N:]:
        ari_dict[distance_metric] = NOT_APPLICABLE

    f_dbs = open(f"./dataset_labels/txt_indiv/{dbs}", "r")
    dbs_dict = f_dbs.read()
    dbs_dict = dbs_dict.replace("'", '"')
    dbs_dict = json.loads(dbs_dict)
    f_dbs.close()
    
    for distance_metric in clustering_results['best_dist_metric_dbs'].value_counts().index[TOP_N:]:
        dbs_dict[distance_metric] = NOT_APPLICABLE

    f_time = open(f"./dataset_labels/txt_indiv/{time_elapsed}", "r")
    time_dict = f_time.read()
    time_dict = time_dict.replace("'", '"')
    time_dict = json.loads(time_dict)
    f_time.close()

    ari_sorted = []
    dbs_sorted = []

    # Convert dictionaries to arrays of the form
    # [key, perfomance, time].
    for key in ari_dict.keys():
        if ari_dict[key] != NOT_APPLICABLE:
            ari_sorted.append([key, ari_dict[key], time_dict[key]])
        if dbs_dict[key] != NOT_APPLICABLE and key != 'standardized_euclidean':
            dbs_sorted.append([key, dbs_dict[key], time_dict[key]])
            
    print(dbs_sorted)
    
    # Sort arrays by performance (best to worst) then by time (fastest to slowest).
    ari_sorted.sort(key=lambda items: (items[1], -items[2]), reverse = True)
    dbs_sorted.sort(key=lambda items: (items[1], items[2]))

    filename, _ = ari.rsplit('_', 1)

    # Get the best distance metric based on ARI
    ari_len = len(ari_sorted)
    if ari_len != 0:
        best_dist_metric_ari = ari_sorted[0][0]
        best_dist_metric_ari_eval = ari_sorted[0][1]
    else:
        best_dist_metric_ari = NOT_APPLICABLE
        best_dist_metric_ari_eval = NOT_APPLICABLE
    
    # Get the best distance metric based on DBS
    dbs_len = len(dbs_sorted)
    if dbs_len != 0:
        best_dist_metric_dbs = dbs_sorted[0][0]
        best_dist_metric_dbs_eval = dbs_sorted[0][1]
    else:
        best_dist_metric_dbs = NOT_APPLICABLE
        best_dist_metric_dbs_eval = NOT_APPLICABLE    
        
    DATA = f"{filename},{best_dist_metric_ari},{best_dist_metric_ari_eval},{best_dist_metric_dbs},{best_dist_metric_dbs_eval}\n"
    f_out.write(DATA)

f_out.close()

[['manhattan', 1.646078756495926, 0.0003126999999949476], ['euclidean', 1.5631141107350428, 0.0005911000000082822], ['chebyshev', 1.6930221252122728, 0.0002513000000021748]]
[['manhattan', 2.1808200807132536, 0.00022849999999152715], ['euclidean', 2.1032708493554537, 0.0007823999999914122], ['chebyshev', 2.6724410180884806, 0.0005093999999985499]]
[['manhattan', 1.7954272655665566, 0.00048300000000267573], ['euclidean', 1.6995370572232396, 0.00020169999999097854], ['chebyshev', 1.3021618833967545, 0.0005204999999932625]]
[['manhattan', 2.1434945749878582, 0.00013919999997824561], ['euclidean', 2.5026912833489154, 0.00017139999999926658], ['chebyshev', 3.29193062953697, 0.00017909999999687898]]
[['manhattan', 1.4182891930178894, 0.0008007000000134212], ['euclidean', 1.359764424296365, 0.00029230000001234657], ['chebyshev', 1.4377211118281037, 0.0003489999999999327]]
[['manhattan', 2.0908212851116668, 0.00023440000001073713], ['euclidean', 1.9906251026764672, 0.0004868999999985135], ['ch

[['manhattan', 2.815529568833748, 0.008205400000178997], ['euclidean', 2.803957441132023, 0.006709699999873919], ['chebyshev', 1.2006620989253969, 0.0062748000000283355]]
[['manhattan', 1.2160339968154399, 7.199999981821747e-05], ['euclidean', 1.2160339968154399, 4.500000000007276e-05], ['chebyshev', 2.032137123714029, 4.73000000056345e-05]]
[['manhattan', 1.1216958516024238, 0.0019418000001678593], ['euclidean', 1.0707723814339092, 0.001750599999922997], ['chebyshev', 1.488005803360689, 0.004280399999970541]]
[['manhattan', 1.234131202928618, 4.81000000718268e-05], ['euclidean', 1.23963200756673, 3.479999998035055e-05], ['chebyshev', 2.6102099282057085, 3.550000008090137e-05]]
[['manhattan', 1.1256411003053197, 9.84000002972607e-05], ['euclidean', 1.1363352910881965, 0.00010000000020227162], ['chebyshev', 1.3609941956795688, 0.00016400000004068715]]
[['manhattan', 1.5470592609730471, 0.0003907000000253902], ['euclidean', 2.1885953501541477, 0.00023600000031365198], ['chebyshev', 2.229

[['manhattan', 1.6352208903948475, 0.0003407000003790017], ['euclidean', 1.6007721081871649, 0.0003330999998070183], ['chebyshev', 2.010360598983815, 0.0004604999994626269]]
[['manhattan', 0.7504185610713139, 0.00011730000005627517], ['euclidean', 0.7504185610713139, 7.329999971261714e-05], ['chebyshev', 0.7504185610713139, 8.309999975608662e-05]]
[['manhattan', 1.3341842189035669, 9.15000000532018e-05], ['euclidean', 0.9787654220400803, 0.000127599999927952], ['chebyshev', 1.0660136250449714, 0.00010829999996531114]]
[['manhattan', 0.5674865521254786, 3.3799999982875306e-05], ['euclidean', 0.5674865521254786, 3.509999987727497e-05], ['chebyshev', 0.5830865191327181, 3.509999987727497e-05]]
[['manhattan', 1.5117030152839517, 0.00012110000000120635], ['euclidean', 1.3459964205619708, 0.0001238999999486623], ['chebyshev', 1.6812391240354494, 0.0001838999999108637]]
[['manhattan', 2.953909589495183, 6.899999993947858e-05], ['euclidean', 2.911289174968647, 0.00021839999999428983], ['chebys

[['manhattan', 3.220000007786439e-05, 5.5499999689345714e-05], ['euclidean', 3.229999992981902e-05, 5.5700000302749686e-05], ['chebyshev', 3.0000000151630957e-05, 5.899999996472616e-05]]
[['manhattan', 2.659012694909104, 5.760000021837186e-05], ['euclidean', 2.633280891004009, 9.040000077220611e-05], ['chebyshev', 0.976347742408163, 5.710000004910398e-05]]
[['manhattan', 0.9146913370339057, 0.001780999999937194], ['euclidean', 0.9188860492140235, 0.0017288999997617793], ['chebyshev', 0.9647467919535356, 0.001808099999834667]]
[['manhattan', 0.44065974453419754, 4.230000001825829e-05], ['euclidean', 0.44065974453419754, 1.7500000012660166e-05], ['chebyshev', 0.44065974453419754, 2.9899999958615808e-05]]
[['manhattan', 1.4188312878598939, 0.0007716000000073109], ['euclidean', 1.524045300654288, 0.0012125000002924935], ['chebyshev', 1.4605636119942031, 0.0008892999994714046]]
[['manhattan', 0.8510074289478964, 1.4900000223860843e-05], ['euclidean', 0.8617242472820775, 1.5000000075815478e-

[['manhattan', 4.027229551337984, 7.860000005166512e-05], ['euclidean', 3.9480453320911995, 3.550000019458821e-05], ['chebyshev', 6.1161872572712195, 3.5599999137048144e-05]]
[['manhattan', 2.354010225402267, 1.5499999790336005e-05], ['euclidean', 3.0369430083042164, 1.4899999769113492e-05], ['chebyshev', 1.0461230756985216, 1.269999938813271e-05]]
[['manhattan', 1.767536467395129, 1.939999947353499e-05], ['euclidean', 1.6926180288936568, 1.9500000234984327e-05], ['chebyshev', 1.104046237152169, 1.8699999600357842e-05]]
[['manhattan', 1.6039177553515955, 1.1899999663000926e-05], ['euclidean', 1.5423512697155082, 1.1199999789823778e-05], ['chebyshev', 0.7826362013404314, 1.089999932446517e-05]]
[['manhattan', 1.2601985581669566, 2.5899999855028e-05], ['euclidean', 1.2584498181522594, 3.170000036334386e-05], ['chebyshev', 1.354691085083018, 2.6599999728205148e-05]]
[['manhattan', 0.891730432609392, 1.2599999536178075e-05], ['euclidean', 0.891730432609392, 1.0600000678095967e-05], ['cheby

[['manhattan', 0.20544379010235553, 0.0001754000004439149], ['euclidean', 0.20544379010235553, 0.00017829999978857813], ['chebyshev', 0.20544379010235553, 0.0002870999996957835]]
[['manhattan', 1.2274368119637338, 0.008779400001003523], ['euclidean', 1.2274368119637338, 0.010324599999876227], ['chebyshev', 1.1227519299870463, 0.026370699999461067]]
[['manhattan', 4.591697957198605, 8.50000014906982e-06], ['euclidean', 4.53984464015592, 8.399999387620483e-06], ['chebyshev', 1.0621558780111555, 7.900000127847306e-06]]
[['manhattan', 0.43982692099453646, 6.949999988137279e-05], ['euclidean', 0.43982692099453646, 7.470000036846614e-05], ['chebyshev', 0.43982692099453646, 0.00014289999944594456]]
[['manhattan', 1.267472762171666, 0.00011470000026747584], ['euclidean', 1.2079440287883356, 8.39000003907131e-05], ['chebyshev', 1.1984819904031176, 7.15000005584443e-05]]
[['manhattan', 1.0555421598560109, 0.0017235999994227313], ['euclidean', 0.6529477407395479, 0.0017256000000998029], ['chebysh

In [73]:
clustering_results = pd.read_csv('dataset_labels/csv_collated/results_top_metrics.csv')
clustering_results

,dataset,best_dist_metric_ari,best_dist_metric_ari_eval,best_dist_metric_dbs,best_dist_metric_dbs_eval
0,a1_raw,manhattan,0.196831,euclidean,1.563114
1,a1_va3,euclidean,0.181099,euclidean,2.103271
2,a2_raw,euclidean,0.297242,chebyshev,1.302162
3,a2_va3,euclidean,0.163337,manhattan,2.143495
4,a3_raw,euclidean,0.287089,euclidean,1.359764
...,...,...,...,...,...
335,winequality-white,euclidean,0.041872,euclidean,1.780022
336,wine,euclidean,0.003037,euclidean,1.348897
337,wisconsin,mahalanobis,0.025664,manhattan,1.682554
338,Zemberek-Stemmed,euclidean,0.001240,manhattan,0.870462


In [74]:
clustering_results['best_dist_metric_ari'].value_counts()

manhattan      130
euclidean      112
mahalanobis     98
Name: best_dist_metric_ari, dtype: int64

In [75]:
clustering_results['best_dist_metric_dbs'].value_counts()

chebyshev    139
euclidean    122
manhattan     79
Name: best_dist_metric_dbs, dtype: int64

<hr>

## References

[1] X. Zhu, Y. Li, J. Wang, T. Zheng, and J. Fu, "Automatic recommendation of a distance measure for
clustering algorithms," *ACM Transactions on Knowledge Discovery from Data*, vol. 15, no. 1, pp. 7-22,
December 2020.

[2] B. A. Pimentel and A. C. P. L. F. de Carvalho, "Statistical versus distance-based meta-features for
clustering algorithm recommendation using meta-learning," in Proc. 2018 International Joint Conference
on Neural Networks (IJCNN), 2018, pp. 1-8.

[3] B. A. Pimentel and A. C. P. L. F. de Carvalho, "A Meta-learning approach for recommending the
number of clusters for clustering algorithms," *Knowledge-Based Systems*, vol. 195, May 2020.

[4] L. Hubert and P. Arabie, "Comparing partitions," *Journal of Classification*, vol. 2, pp. 193-218, December 1985.

[5] D. L. Davies and D. W. Bouldin, "A cluster separation measure," *IEEE Transactions on Pattern Analysis and Machine Intelligence*, vol. PAMI-1, no. 2, pp. 224-227, April 1979.